In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys

sys.path.append(os.path.abspath('../..'))
from modules import io
import modules.vascular_data as sv
from scipy.interpolate import Rbf, NearestNDInterpolator, LinearNDInterpolator
import vtk

In [2]:
#CASE = "0110"

CASE = "0144"

#CASE = "coronary"

gen  = 2

SIM = "sim_wom_rcr"
#SIM = "sim_cor_inflow"

CSV = "/media/marsdenlab/Data1/UQ/"+CASE+"/"+SIM+"/csv/"+str(gen)+"_edit.csv"

FIG_DIR  = "/media/marsdenlab/Data1/UQ/"+CASE+"/figures/3dplots"

BASE_VTU = "/media/marsdenlab/Data1/UQ/"+CASE+"/generations/\
1/coarse/0/exterior.vtp"

io.mkdir(FIG_DIR)

quants = [
    "area",
    "radius_actual",
    "pressure_0",
    "vWSS_3_boundary",
    "velocity_3"
]

labels = [
    "area",
    "radius",
    "pressure",
    "WSS",
    "velocity"
]


df = pd.read_csv(CSV)
paths = df['path'].unique()

df_group = df.groupby(['path','point','model']).mean()

In [3]:
df_group.min()

Unnamed: 0                484.000000
Unnamed: 0.1               25.000000
Unnamed: 0.1.1             25.000000
area                        0.098458
generation                  1.000000
length                      1.125099
nx                         -0.936980
ny                         -0.918594
nz                         -0.999724
pressure_0                 82.170738
pressure_0_boundary    109466.639136
radius_actual               0.177031
radius_supplied             1.000000
time                     1750.000000
vWSS_0                     -9.068940
vWSS_0_boundary          -159.845119
vWSS_1                    -11.800540
vWSS_1_boundary          -198.310110
vWSS_2                     -7.320285
vWSS_2_boundary          -191.858171
vWSS_3                      0.024238
vWSS_3_boundary             2.012910
velocity_0                -76.854177
velocity_0_boundary        -9.407295
velocity_1                -72.869215
velocity_1_boundary       -12.157680
velocity_2                -77.110388
v

In [4]:
vtu = sv.vtk_read_native_polydata(BASE_VTU)

mu  = df_group.groupby(["path","point"]).mean()
std = df_group.groupby(["path","point"]).std()
print(mu['vWSS_3_boundary'])
print(std)

path         point
SMA          5         18.191060
             6         20.175448
             7         27.219161
             8         35.000607
             9         47.954725
             10        59.580781
             11        57.782916
             12        51.866781
             13        45.235637
             14        39.699359
             15        35.983053
             16        34.707719
             17        34.428791
             18        32.522997
             19        27.444520
             20        22.479672
             21        21.212313
             22        23.650783
             23        27.555261
             24        27.792508
             25        23.112602
             26        18.998282
             27        17.942212
             28        19.359253
             29        21.613841
             30        23.809160
             31        25.744892
             32        28.165386
             33        30.101998
             34        3

In [5]:
S1 = 0
S2 = -1
EPS = 0.01

x = mu['x'].values[S1:S2,np.newaxis]
y = mu['y'].values[S1:S2,np.newaxis]
z = mu['z'].values[S1:S2,np.newaxis]

points = np.hstack((x,y,z))

for j,q in enumerate(quants):
    print(q)
    v1 = std[q].values[S1:S2]
    v2 = v1/(mu[q].values[S1:S2]+EPS)
    v3 = (v1-np.amin(v1))/(np.amax(v1)-np.amin(v1)+EPS)
    v4 = mu[q].values[S1:S2]
    print(q,np.amin(v4),np.amax(v4))
#     interp1 = Rbf(x,y,z,v1, function='gaussian',epsilon=0.5)
#     interp2 = Rbf(x,y,z,v2, function='gaussian',epsilon=0.5)
#     interp3 = Rbf(x,y,z,v3, function='gaussian',epsilon=0.5)

    interp1 = NearestNDInterpolator(points,v1)
    interp2 = NearestNDInterpolator(points,v2)
    interp3 = NearestNDInterpolator(points,v3)
    interp4 = NearestNDInterpolator(points,v4)

#     interp1 = LinearNDInterpolator(points,v1)
#     interp2 = LinearNDInterpolator(points,v2)
#     interp3 = LinearNDInterpolator(points,v3)


    for interp,label in zip([interp1,interp2,interp3,interp4],\
                            [' std',' cv',' std norm',' mean']):
        array = vtk.vtkDoubleArray()
        array.SetNumberOfComponents(1)
        array.SetNumberOfTuples(vtu.GetNumberOfPoints())
        for i in range(vtu.GetNumberOfPoints()):
            point = vtu.GetPoint(i)
            v = interp(*point)
            array.SetValue(i, v)

        vtu.GetPointData().AddArray(array)
        array.SetName(labels[j]+label)

sv.vtk_write_native_polydata(vtu,FIG_DIR+'/{}.vtp'.format(gen))

area
area 0.11835702487447827 7.834839827831178
radius_actual
radius_actual 0.1939769099955944 1.5791626578795885
pressure_0
pressure_0 84.61693436602883 101.10930683799135
vWSS_3_boundary
vWSS_3_boundary 2.6895310690988943 235.00557466211458
velocity_3
velocity_3 9.052072350384668 97.39273748746
